In [106]:
from simpleai.search import CspProblem, backtrack
puzzle = input("Give a cryptarithmetic puzzle.")



The next step is to define a few things:
1. The operator ( +, -, *, / )
2. We need the 3 words, the first 2 words in the calculation and the result word.

To find the operator we first need to define what operations we want to be possible. In the exercises, there was + and *, but for this task I want to include - and /.

For the words we slice the given puzzle. We will define a correct form of puzzle as 'word1 + word2 = result', so between the 2 first there is an operator and in between the 2 last words there is an equal sign. Because we know this, we can slice the puzzle by finding the operator and the equal sign. After slicing the puzzle we remove the spaces and we capitalize the letters.

In [107]:
print (puzzle)

if puzzle.find("+") != -1:
    operator = "+"
elif puzzle.find("-") != -1:
     operator = "-"
elif puzzle.find("*") != -1:
     operator = "*"
elif puzzle.find("/") != -1:
     operator = "/"
else:
    print("The given puzzle is not in a correct form.")
print (operator)


word_1 = puzzle[:puzzle.index(operator)].replace(" ","").upper()
word_2 = puzzle[puzzle.index(operator)+1:puzzle.index("=")].replace(" ","").upper()
word_result = puzzle[puzzle.index("=")+1:].replace(" ","").upper()


print(word_1)
print(word_2)

print(word_result)


to + go = out
+
TO
GO
OUT


Next we create a tuple of all the unique letters. Also a dictionairy where each possible number is appointed to a letter. Of course the first letters of the words cannot have 0 as a value.

In [108]:

variables = []
for word in (word_1, word_2, word_result):
    for char in word:
        if char not in variables:
            variables.append(char)

variables = tuple(variables)
print(variables)
domains = {}

for letter in variables:
    if letter in [word_1[0], word_2[0], word_result[0]]:
        domains[letter] = list(range(1, 10))
    else:
        domains[letter] = list(range(0, 10))
print(domains)

('T', 'O', 'G', 'U')
{'T': [1, 2, 3, 4, 5, 6, 7, 8, 9], 'O': [1, 2, 3, 4, 5, 6, 7, 8, 9], 'G': [1, 2, 3, 4, 5, 6, 7, 8, 9], 'U': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]}


In [109]:
def constraint_unique(variables, values):
    return len(values) == len(set(values))
def constraint_calculation(variables, values):
    number_1 = ""
    number_2 = ""
    result=""



    for letter in word_1:
        number_1+=str(values[variables.index(letter)])
    number_1 = int(number_1)
    
    for letter in word_2:
        number_2+=str(values[variables.index(letter)])
    number_2 = int(number_2)


    for letter in word_result:
        result+=str(values[variables.index(letter)])
    result = int(result)


    if operator ==  "+":
        return (number_1 + number_2) == result
    elif operator == "-":
        return result == (number_1 - number_2) == result
    elif operator == "*":
        return (number_1 * number_2) == result
    else:
        return (number_1 / number_2) ==result

constraints = [
    (variables, constraint_unique),
    (variables, constraint_calculation),
]

problem = CspProblem(variables, domains, constraints)

output = backtrack(problem)
print('\nSolutions:', output)


Solutions: {'T': 2, 'O': 1, 'G': 8, 'U': 0}
